In [1]:
from datasets import load_dataset

dataset = load_dataset("openbmb/UltraInteract_pair", split="train")

/home/heekue83/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 100%|██████████| 219522/219522 [00:01<00:00, 143839.44 examples/s]
/home/heekue83/.local/lib/python3.10/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [21]:
df = dataset.to_pandas()
df["depth"] = df.trajectory.apply(lambda x: len(x))
df["parent_response"] = df.trajectory.apply(lambda x: x[-2]['value'] if len(x) > 1 else None)
display(df.head())
# df = df[df.trajectory.apply(lambda x: len(x) == 3)]
# df.shape

,task,dataset,trajectory,chosen,rejected,id,parent_id,depth,parent_response
0,Coding,codecontest,"[{'from': 'user', 'value': 'Write Python code ...",Step 1: This problem asks us to determine if ...,Step 1: We need to determine if it is possibl...,390473b96c1696b6dda5bc62c6f6124651c7cf327c80df...,f0c215931a9f22324fc897d6d2fbe6ccb18688bf17cea6...,1,None
1,Coding,codecontest,"[{'from': 'user', 'value': 'Write Python code ...","Step 1: To solve this problem, we need to che...",Step 1: We need to check if it's possible to ...,b5a8357b77d7599adf256586b94eaa76b77aecb106e20d...,f0c215931a9f22324fc897d6d2fbe6ccb18688bf17cea6...,1,None
2,Coding,codecontest,"[{'from': 'user', 'value': 'Write Python code ...","Step 1: To solve this problem, we need to che...",Step 1: We need to determine if we can change...,ab7a69d361f06c74a45c56f1574ff0d8ab7ca83265b8a0...,f0c215931a9f22324fc897d6d2fbe6ccb18688bf17cea6...,1,None
3,Coding,codecontest,"[{'from': 'user', 'value': 'Write Python code ...","Step 1: To solve this problem, we need to det...","Step 1: To solve this problem, we need to che...",4f760beff13ec9f6ea25096dcec05d29d6474c65bb3c57...,f0c215931a9f22324fc897d6d2fbe6ccb18688bf17cea6...,1,None
4,Coding,codecontest,"[{'from': 'user', 'value': 'Write Python code ...","Step 1: To solve this problem, we need to det...",Step 1: We need to determine if we can change...,c561cfe06fa7057aa517a2175a969e03d4e1626ac9250e...,f0c215931a9f22324fc897d6d2fbe6ccb18688bf17cea6...,1,None


In [22]:
df.parent_id.unique().shape

(64700,)

In [23]:
df.trajectory.apply(len).value_counts(normalize=True)

trajectory
1     0.726105
3     0.117346
5     0.069401
7     0.048660
9     0.038484
11    0.000005
Name: proportion, dtype: float64

In [24]:
sample1 = df[df.depth == 3].sample(1)
print(sample1.parent_id.values[0])
sample1

ba1194c98df0b3e32560155074c5b1ca8ed80601b935bd177df550e17caf859d


,task,dataset,trajectory,chosen,rejected,id,parent_id,depth,parent_response
219223,Logic,strategyqa,"[{'from': 'user', 'value': 'Tool available: [1...","Step 1: From the information we found, the Pal...",Step 1: I will guess that Cookie Monster's die...,ba1194c98df0b3e32560155074c5b1ca8ed80601b935bd...,ba1194c98df0b3e32560155074c5b1ca8ed80601b935bd...,3,Step 1: Let me check if Cookie Monster's main ...


In [27]:
df[sample1.parent_id.values[0] == df.parent_id]
# sample1.trajectory.values[0]

,task,dataset,trajectory,chosen,rejected,id,parent_id,depth,parent_response
219223,Logic,strategyqa,"[{'from': 'user', 'value': 'Tool available: [1...","Step 1: From the information we found, the Pal...",Step 1: I will guess that Cookie Monster's die...,ba1194c98df0b3e32560155074c5b1ca8ed80601b935bd...,ba1194c98df0b3e32560155074c5b1ca8ed80601b935bd...,3,Step 1: Let me check if Cookie Monster's main ...


In [33]:
# df[df.depth == 1].shape
# df[df.depth == 3].parent_id.unique().shape

(25760, 9)

In [30]:
df[(df.parent_id == "f0c215931a9f22324fc897d6d2fbe6ccb18688bf17cea67c26d259b679173521") & (df.depth >= 3)].parent_id.values

array(['f0c215931a9f22324fc897d6d2fbe6ccb18688bf17cea67c26d259b679173521',
       'f0c215931a9f22324fc897d6d2fbe6ccb18688bf17cea67c26d259b679173521',
       'f0c215931a9f22324fc897d6d2fbe6ccb18688bf17cea67c26d259b679173521'],
      dtype=object)

In [42]:
subtrees = []

for parent_id, subdf in df.groupby("parent_id"):
    if subdf.depth.max() == 1:
        continue

    for i, row in subdf.iterrows():
        for j, child in subdf[subdf.depth > row.depth].iterrows():
            if row.rejected == child.parent_response:
                # print("===" * 15)
                # print(row)
                # print(child)
                # print()

                subtree = {
                    "parent": row.to_dict(),
                    "child": child.to_dict()
                }
                subtree['parent'].pop("parent_response")
                subtree['child'].pop("parent_response")
                subtrees.append(subtree)
    
subtrees[0]

{'parent': {'task': 'Coding',
  'dataset': 'TACO',
  'trajectory': array([{'from': 'user', 'value': 'Write Python code to solve the task:\nYour task is to write a function that does just what the title suggests (so, fair warning, be aware that you are not getting out of it just throwing a lame bas sorting method there) with an array/list/vector of integers and the expected number `n` of smallest elements to return.\n\nAlso:\n\n* the number of elements to be returned cannot be higher than the array/list/vector length;\n* elements can be duplicated;\n* in case of duplicates, just return them according to the original order (see third example for more clarity).\n\nSame examples and more in the test cases:\n\n```python\nfirst_n_smallest([1,2,3,4,5],3) == [1,2,3]\nfirst_n_smallest([5,4,3,2,1],3) == [3,2,1]\nfirst_n_smallest([1,2,3,4,1],3) == [1,2,1]\nfirst_n_smallest([1,2,3,-4,0],3) == [1,-4,0]\nfirst_n_smallest([1,2,3,4,5],0) == []\n```\n\n[Performance version by FArekkusu](https://www.cod

In [43]:
len(subtrees)

59603

In [44]:
from datasets import Dataset

subtree_ds = Dataset.from_list(subtrees)
subtree_ds.push_to_hub("heegyu/UltraInteract_pair_subtree")

Pushing dataset shards to the dataset hub: 100%|██████████| 2/2 [00:08<00:00,  4.33s/it]
